In [1]:
import pandas as pd

In [6]:
movies=pd.read_csv("G:/csv_files/ml-latest-small/movies.csv")
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings=pd.read_csv("G:/csv_files/ml-latest-small/ratings.csv")
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [21]:
ratings.rating.unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [16]:
ratings["rate_count"]=1

In [15]:
ratings.shape

(100836, 4)

In [20]:
rate_count_df=ratings.groupby("movieId").count()["rate_count"].reset_index()
rate_count_df.head(8)

,movieId,rate_count
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49
5,6,102
6,7,54
7,8,8


In [26]:
df=ratings.merge(rate_count_df,how="left",on="movieId").drop(["timestamp","rate_count_x"],axis=1)
df.head(10)

,userId,movieId,rating,rate_count_y
0,1,1,4.0,215
1,1,3,4.0,52
2,1,6,4.0,102
3,1,47,5.0,203
4,1,50,5.0,204
5,1,70,3.0,55
6,1,101,5.0,23
7,1,110,4.0,237
8,1,151,5.0,44
9,1,157,5.0,11


In [28]:
df=df[df["rate_count_y"]>50]

In [31]:
df=df.pivot(index="movieId",columns="userId",values="rating").fillna(0)

In [33]:
df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0
7,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0


In [34]:
from scipy.sparse import csr_matrix  #compressed row matrix

In [37]:
final_df=csr_matrix(df.to_numpy())

# BASED ON COSINE SIMILARITY

In [39]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric="cosine",algorithm='brute', n_neighbors=5)

In [40]:
model.fit(final_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [99]:
# DEPLOYMENT OF MODEL

In [100]:
def get_similar_movies(index):
    distance,similar_movie=model.kneighbors(X=[list(df.iloc[index].to_numpy())], n_neighbors=6)
    print("THE MOVIE NAME IS :",movies.iat[similar_movie[0][0],1])
    print("\n\nTHE RECOMMENDATION MOVIE FOR THE USER IS ")
    for i in similar_movie[0][1:]:
        
        print("-->\t",movies.iat[i,1])

In [105]:
movieid=[*range(1,4)]
for i in movieid:
    get_similar_movies(i)
    print("\n\n")

THE MOVIE NAME IS : Jumanji (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 Fair Game (1995)
-->	 Vampire in Brooklyn (1995)
-->	 Kicking and Screaming (1995)
-->	 Nick of Time (1995)
-->	 Bridges of Madison County, The (1995)



THE MOVIE NAME IS : Grumpier Old Men (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 Birdcage, The (1996)
-->	 Jupiter's Wife (1994)
-->	 Copycat (1995)
-->	 Flirting With Disaster (1996)
-->	 Father of the Bride Part II (1995)



THE MOVIE NAME IS : Waiting to Exhale (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 NeverEnding Story III, The (1994)
-->	 Balto (1995)
-->	 Lamerica (1994)
-->	 Tom and Huck (1995)
-->	 Steal Big, Steal Little (1995)





# BASED ON EULER DISTANCE

In [112]:
model2=NearestNeighbors(metric="euclidean",algorithm='brute', n_neighbors=5)
model2.fit(final_df)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [113]:
def get_similar_movies(index):
    distance,similar_movie=model2.kneighbors(X=[list(df.iloc[index].to_numpy())], n_neighbors=6)
    print("THE MOVIE NAME IS :",movies.iat[similar_movie[0][0],1])
    print("\n\nTHE RECOMMENDATION MOVIE FOR THE USER IS ")
    for i in similar_movie[0][1:]:
        
        print("-->\t",movies.iat[i,1])

In [114]:
movieid=[*range(1,4)]
for i in movieid:
    get_similar_movies(i)
    print("\n\n")

THE MOVIE NAME IS : Jumanji (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
-->	 GoldenEye (1995)
-->	 Star Wars: Episode IV - A New Hope (1977)
-->	 Dangerous Minds (1995)
-->	 Postman, The (Postino, Il) (1994)



THE MOVIE NAME IS : Grumpier Old Men (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 Juror, The (1996)
-->	 Man of the Year (1995)
-->	 Father of the Bride Part II (1995)
-->	 Birdcage, The (1996)
-->	 Crossing Guard, The (1995)



THE MOVIE NAME IS : Waiting to Exhale (1995)


THE RECOMMENDATION MOVIE FOR THE USER IS 
-->	 Copycat (1995)
-->	 Babe (1995)
-->	 Up Close and Personal (1996)
-->	 Tom and Huck (1995)
-->	 Wild Bill (1995)



